## Script to load bundle and unbundle using dict

for some reason the pyFHIRR4 models not working :-(

- python version 3.6+
- get data from folder
- unbundle
- display


In [1]:
from os import listdir
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhirclient.r4models import bundle as B
from fhirclient.r4models.fhirabstractbase import FHIRValidationError

### globals

In [2]:
 

types = dict (
Patient = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
AllergyIntolerance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
CarePlan = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
CareTeam = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
Condition = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
Device = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-device',
DiagnosticReport = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport',
DocumentReference = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
Encounter = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
Goal = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
Immunization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
Location = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
Medication = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
MedicationStatement = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationstatement',
MedicationRequest = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
Observation = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observationresults',
'http://hl7.org/fhir/StructureDefinition/vitalsigns',
'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
],
Organization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
Practitioner = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
PractitionerRole = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
Procedure = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
)

r_list =[]

#in_path = '/Users/ehaas/Documents/FHIR/synthea/output/fhir/'
in_path='test/'
#f_name = 'Denisse335_Stracke611_e760e08d-a0ff-4cd7-83c0-36b27dae6ec3'
#f_name='Alaina222_Gleichner915_0e83bdc3-52c5-4e7e-ba92-174e711ba95c'



### Get files

In [4]:
def open_file(in_path,f_name): # get files
    with open(f'{in_path}{f_name}',encoding='utf-8', errors='ignore') as f:
        r = f.read()
       
        return(loads(r))

### Write file to examples dir in IG

In [5]:
def write_file(r_json,type,id): # write file
    out_path = '/Users/ehaas/Documents/FHIR/USCoreR4Validator/source/examples/'  # append forward slash   
    # out_path=''
    with open(f'{out_path}{type}-{id}.json', 'w') as f:
        f.write(r_json)

### Loop through all entries and if US Core type change logical to literal reference and save

#### Loop through entries and create literal to logical reference map

In [6]:
def select_entries(b):
    global r_list
    log2lit = {e['fullUrl']:f'{e["resource"]["resourceType"]}/{e["resource"]["id"]}' for e in b["entry"]} 
    for e in b["entry"]:
        r = e["resource"]
        t = r["resourceType"]
        i = r["id"]
        if t in types.keys(): 
           r_list.append(t)
           if "Observation" not in t : # add meta profiles
               r['meta']= dict(profile =[types[t]])
           else:  # "Observation" 
               #['vital-signs','laboratory','72166-2']:
               if 'laboratory' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][0]])
               elif 'vital-signs' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][1]])
               elif '72166-2' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][2]])
               else:  # no profile
                   pass
           r_json = dumps(r, indent=4) 
           for k,v in log2lit.items(): #  change logical references to literal references and write to examples files for processing.
               r_json = r_json.replace(k,v) 
           print(f' writing resource {t}-{i} to examples folder' )
           write_file(r_json,t,i)

        """
        for e in b["entry"]:
        r = e["resource"]
        t = r["resourceType"]
        i = r["id"]
        if t == "Observation" and 'vital' in r['category'][0]['coding'][0]['code']:
            print(f'vitals observation resource  = {t}-{i}')
            print(dumps(r, indent=4))
        elif t == "Observation" and 'lab' in r['category'][0]['coding'][0]['code']:
            print(f'lab result resource  = {t}-{i}')
            print(dumps(r, indent=4))"""

## Main File call all the functions above

In [7]:
for file in listdir(in_path):
    if file.endswith(".json"):
        print("json file found:\t", file)
        b = open_file(in_path,file)
        select_entries(b)
print(f'number of example resource = {len(r_list)}')             
print(f'US Core types = {types.keys()}')
print(f'Synthea types {set(r_list)}')            
print(f'What is missing {set(types)-set(r_list)}')

print('======Fin======')

json file found:	 Alaina222_Gleichner915_0e83bdc3-52c5-4e7e-ba92-174e711ba95c.json
 writing resource Patient-42a853f4-73ea-4958-afb5-6809221ff231 to examples folder
 writing resource Organization-3bd5eda0-16da-3ba5-8500-4dfd6ae118b8 to examples folder
 writing resource Practitioner-7a720384-0885-4bd7-8496-492f55ca5cc7 to examples folder
 writing resource Encounter-af729e9f-ab61-4300-8536-7ca1601b25e1 to examples folder
 writing resource Condition-12488ebe-820f-4c70-ac79-8f553d846775 to examples folder
 writing resource Goal-bb9f7d33-b0c4-4159-a939-aebcf61c7bba to examples folder
 writing resource Goal-5746e003-6d9c-477d-88ae-0ef30d09891a to examples folder
 writing resource Goal-2db5a192-213d-40e5-a46a-9d77c083e288 to examples folder
 writing resource Goal-ed858521-9d90-471f-87ae-09f49a3f5251 to examples folder
 writing resource Goal-a8aca83a-267f-4b9d-ae22-7ea4fb858929 to examples folder
 writing resource CarePlan-eb579a42-36ca-4125-ab51-011f434c1d8e to examples folder
 writing resour